# Welcome to MICE tutorial

Why Multiple Imputation Chained Equation?

Scenario - Data for “predicting employee happiness 😁 index” has some missing employment status (0-100% FTE) and salary attributes along with job title, location, team and function. You need to impute those missing values.
Standard impute strategies would simply take the mean or median producing insignificant results. 
MICE performs a special imputation that uses as much information as possible from the other features (independent variables) to impute depending on each employee feature.



Link to the [MICE Github](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/imputation/mice.py)

## Prepare the notebook

In [103]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.utils import check_array
from time import time

from statsmodels.imputation.mice import MICEData
from statsmodels.imputation.mice import MICE as mice
from statsmodels.regression.linear_model import OLS
from MICE import MiceImputer
%matplotlib inline

## Load the data

In [60]:
train = pd.read_csv("smalldata.csv")
train['index'] = train['Unnamed: 0']
train = train.drop('Unnamed: 0', axis = 1)
train.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,index
0,NO_LABEL,NO_LABEL,NO_LABEL,Non-School,NO_LABEL,NO_LABEL,Supplies/Materials,NO_LABEL,PreK-12 Operating,SUPPLIES,...,NaN,NaN,INSTRUCTION,NaN,NaN,534.00000,HIGH SCHOOL GENERAL EDUCATION PROGRAM 9-12,General Fund,NaN,414351
1,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,EMPLOYEE BENEFITS,...,PERSONNEL-PAID LEAVE,NaN,NON-PROJECT,NaN,ADMINISTRATOR,NaN,STAFF SERVICES,NaN,CENTRAL,156609
2,Special Population Program Management & Support,ISPD,Leadership & Management,Non-School,Special Education,Other,Other Compensation/Stipend,NO_LABEL,PreK-12 Operating,WORKSHOP PARTICIPANT,...,ESE,NaN,INST STAFF TRAINING SVCS,NaN,NaN,382.72000,NaN,FED THRU STATE-CASH ADVANCE,IDEA PART B,75964
3,Teacher Compensation,Instruction,School Reported,School,ELL,Teacher,Base Salary/Compensation,NO_LABEL,PreK-12 Operating,NaN,...,NaN,1.0,NaN,NaN,NaN,64644.64165,NaN,NaN,NaN,172213
4,Extended Time & Tutoring,Instruction,School Reported,School,Unspecified,Teacher,Other Compensation/Stipend,Non PreK,PreK-12 Operating,Salaries And Wages For Teachers And Other Prof...,...,School,NaN,Social Work Services,Instruction And Curriculum,TEACHER BACHELOR,78.71418,Undistributed,Title Iv Part B Century Community Learning Ce,EXTENDED DAYS,348071


bascially get all the stuff from our main notebook here and run using miceData

## Data Cleaning

* In this step, We are removing the white strips and capitalizing our categorical variables 

In [61]:
train_trimmed = train.apply(lambda x: x.str.strip() if x.dtype == "object" else x) #change to category if ran 
#the cell which changes object to category
train_trimmed = train_trimmed.apply(lambda x: x.str.upper() if x.dtype == "object" else x)
train_trimmed.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,index
0,NO_LABEL,NO_LABEL,NO_LABEL,NON-SCHOOL,NO_LABEL,NO_LABEL,SUPPLIES/MATERIALS,NO_LABEL,PREK-12 OPERATING,SUPPLIES,...,NaN,NaN,INSTRUCTION,NaN,NaN,534.00000,HIGH SCHOOL GENERAL EDUCATION PROGRAM 9-12,GENERAL FUND,NaN,414351
1,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NON-OPERATING,EMPLOYEE BENEFITS,...,PERSONNEL-PAID LEAVE,NaN,NON-PROJECT,NaN,ADMINISTRATOR,NaN,STAFF SERVICES,NaN,CENTRAL,156609
2,SPECIAL POPULATION PROGRAM MANAGEMENT & SUPPORT,ISPD,LEADERSHIP & MANAGEMENT,NON-SCHOOL,SPECIAL EDUCATION,OTHER,OTHER COMPENSATION/STIPEND,NO_LABEL,PREK-12 OPERATING,WORKSHOP PARTICIPANT,...,ESE,NaN,INST STAFF TRAINING SVCS,NaN,NaN,382.72000,NaN,FED THRU STATE-CASH ADVANCE,IDEA PART B,75964
3,TEACHER COMPENSATION,INSTRUCTION,SCHOOL REPORTED,SCHOOL,ELL,TEACHER,BASE SALARY/COMPENSATION,NO_LABEL,PREK-12 OPERATING,NaN,...,NaN,1.0,NaN,NaN,NaN,64644.64165,NaN,NaN,NaN,172213
4,EXTENDED TIME & TUTORING,INSTRUCTION,SCHOOL REPORTED,SCHOOL,UNSPECIFIED,TEACHER,OTHER COMPENSATION/STIPEND,NON PREK,PREK-12 OPERATING,SALARIES AND WAGES FOR TEACHERS AND OTHER PROF...,...,SCHOOL,NaN,SOCIAL WORK SERVICES,INSTRUCTION AND CURRICULUM,TEACHER BACHELOR,78.71418,UNDISTRIBUTED,TITLE IV PART B CENTURY COMMUNITY LEARNING CE,EXTENDED DAYS,348071


## Imputation using statsmodel

MICE stands for Multiple Imputation with Chained Equations. We know that FTE and Total Budget are document specific, i.e. a full-time employee and a higher paid employee would not have the same role, designation, operation status, etc as a part-time employee hired on a contract basis. We cannot impute them by taking a mean or median of the entire dataset. Hence, we need a special imputation method that can use as much information as possible from the other features (independent variables) to impute depending on each document (employee) feature. 

In [62]:
# Drop our target features
train_imp = train_trimmed.drop(['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type',
       'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status'],axis=1)
train_imp.head()

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,index
0,SUPPLIES,NaN,HIGH SCHOOL GENERAL ED PROGRAM 9-12,NaN,NaN,NaN,NaN,NaN,NaN,INSTRUCTION,NaN,NaN,534.00000,HIGH SCHOOL GENERAL EDUCATION PROGRAM 9-12,GENERAL FUND,NaN,414351
1,EMPLOYEE BENEFITS,UNASSIGNED PERS/PAID LEAVES,GENERAL FUND,PRINCIPAL,NaN,NaN,NaN,PERSONNEL-PAID LEAVE,NaN,NON-PROJECT,NaN,ADMINISTRATOR,NaN,STAFF SERVICES,NaN,CENTRAL,156609
2,WORKSHOP PARTICIPANT,NaN,NaN,ESE SCHOOL SPECIALIST,NaN,NaN,NaN,ESE,NaN,INST STAFF TRAINING SVCS,NaN,NaN,382.72000,NaN,FED THRU STATE-CASH ADVANCE,IDEA PART B,75964
3,NaN,ESL/BILINGUAL EDUCATION,LOCAL,TEACHER,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,64644.64165,NaN,NaN,NaN,172213
4,SALARIES AND WAGES FOR TEACHERS AND OTHER PROF...,NaN,CASE PARTNERSHIP PROJECT,"TEACHER, BILINGUAL EDUCATION",NaN,UNDISTRIBUTED,EXTENDED DAY,SCHOOL,NaN,SOCIAL WORK SERVICES,INSTRUCTION AND CURRICULUM,TEACHER BACHELOR,78.71418,UNDISTRIBUTED,TITLE IV PART B CENTURY COMMUNITY LEARNING CE,EXTENDED DAYS,348071


In [63]:
train_imp.isnull().sum()

Object_Description         2
Text_2                    19
SubFund_Description        6
Job_Title_Description      5
Text_3                    19
Text_4                    17
Sub_Object_Description    17
Location_Description      15
FTE                       15
Function_Description       4
Facility_or_Department    18
Position_Extra             7
Total                      1
Program_Description        4
Fund_Description          10
Text_1                     7
index                      0
dtype: int64

In [64]:
import re
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in train_imp.columns:
    #  Removing characters other than alphabets
    if train_imp[column_name].dtype == object:
        temp = train_imp[column_name].astype(str).map(lambda x: re.sub(r'\W+', '', x))        
        train_imp[column_name] = le.fit_transform(temp)
    else:
        pass

#### Using default mean/median

In [76]:
train_wo_mice = train_imp.fillna(train_imp.mean())
train_wo_mice

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,index
0,12,3,2,15,3,5,5,4,0.730088,2,3,8,534.000000,5,1,8,414351
1,2,2,1,3,3,5,5,1,0.730088,4,3,0,14280.675006,10,8,0,156609
2,14,3,7,1,3,5,5,0,0.730088,3,3,8,382.720000,13,0,4,75964
3,15,0,4,4,3,5,5,4,1.000000,7,3,8,64644.641650,13,8,8,172213
4,9,3,0,5,3,4,0,2,0.730088,6,2,5,78.714180,12,7,2,348071
5,1,3,1,9,3,5,5,4,0.730088,4,3,7,-63.110000,2,8,1,446304
6,11,3,1,9,0,5,5,4,1.004141,4,3,4,52414.464330,3,8,5,66302
7,4,3,6,15,3,5,2,4,0.730088,5,3,8,-1062.560000,13,6,7,325427
8,1,3,1,2,3,5,5,4,0.730088,4,3,7,-15.020000,2,8,1,49981
9,6,3,7,0,3,3,5,4,0.952253,7,3,2,26639.987380,6,3,8,29225


#### Using set_imputer

In [101]:
imp = MICEData(train_imp)
imp.set_imputer('FTE', formula='Object_Description'+'Function_Description'+'Total')
imp.data.FTE

0     0.149912
1     0.149912
2     0.149912
3     1.000000
4     0.149912
5     0.149912
6     1.004141
7     0.149912
8     0.149912
9     0.952253
10    0.149912
11    0.004310
12    0.149912
13    1.000000
14    0.149912
15    0.149912
16    0.149912
17    0.149912
18    0.149912
19    1.000000
20    0.149912
21    0.880000
22    0.000000
Name: FTE, dtype: float64

#### Using default method

In [71]:
imp2 = MICEData(train_imp)
imp2.data.to_csv

<bound method DataFrame.to_csv of     Object_Description  Text_2  SubFund_Description  Job_Title_Description  \
0                   12       3                    2                     15   
1                    2       2                    1                      3   
2                   14       3                    7                      1   
3                   15       0                    4                      4   
4                    9       3                    0                      5   
5                    1       3                    1                      9   
6                   11       3                    1                      9   
7                    4       3                    6                     15   
8                    1       3                    1                      2   
9                    6       3                    7                      0   
10                   2       3                    1                      9   
11                  10       1

## Using MICE.py (Homegrown algorithm)

In [104]:
mice_imp = MiceImputer(seed_strategy='most_frequent')

In [105]:
fitted_val = mice_imp.fit(train_imp)

In [106]:
miced_data = fitted_val.transform(train_imp)

In [114]:
train_imp[['FTE','Total']].head()

,FTE,Total
0,NaN,534.00000
1,NaN,NaN
2,NaN,382.72000
3,1.0,64644.64165
4,NaN,78.71418


In [113]:
miced_data[['FTE','Total']].head()

,FTE,Total
0,0.678208,534.000000
1,-0.112092,-41097.959158
2,2.606419,382.720000
3,1.000000,64644.641650
4,0.335888,78.714180


**Resources:**

1) https://www.statsmodels.org/dev/generated/statsmodels.imputation.mice.MICEData.html <br>
2) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/ <br>
3) https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87

